In [0]:
import os
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split 
from keras.initializers import RandomNormal
from keras.layers import Dense, Input, Reshape, Dropout, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df2 = pd.read_csv('/content/drive/My Drive/FinalProject822/peptide_data/HLA_peptide_allele_tissue.tsv',delimiter='\t')
# np.random.seed(1337)
noise_dim = 50
batch_size = 64

epochs = 100
optimizer = Adam(0.0002, 0.5)
df2

,peptide_sequence_id,peptide_sequence,tissue_name,hla_type_class,hla_type_name,predicted_affinity
0,7079,FSRFLNKQPY,Kidney,class1,B*35:01,weak
1,10038,ALFDGDPHL,Adrenal Gland,class1,C*03:04,weak
2,3248,KQYERSIVDY,Lung,class1,B*15:01,strong
3,1888,YQTAISENY,Uterus,class1,B*35:01,weak
4,12356,YYIRETENF,Heart,class1,C*06:02,weak
...,...,...,...,...,...,...
449400,30904,GEWDKALSI,Spleen,class1,B*49:01,strong
449401,7850,YFRDQEGQDV,Bone Marrow,class1,C*04:01,strong
449402,50046,KPPKPVSKMRMATPLLMQALP,Tongue,class2,DQA1*01:02-DQB1*03:03,weak
449403,50046,KPPKPVSKMRMATPLLMQALP,Esophagus,class2,DRB4*01:01,strong


In [0]:
encoder_tissue = LabelBinarizer()
train_tissue = df2['tissue_name']
train_tissue = encoder_tissue.fit_transform(train_tissue)

# train_x_2.describe()
# encoder.inverse_transform(train_x_2[:100])
# train_x_2

In [0]:
encoder_class = LabelBinarizer()
train_class = df2['hla_type_class']
train_class = encoder_class.fit_transform(train_class)

In [0]:
encoder_name = LabelBinarizer()
train_name = df2['hla_type_name']
train_name = encoder_name.fit_transform(train_name)


In [0]:
encoder_affinity = LabelBinarizer()
train_affinity = df2['predicted_affinity']
train_affinity = encoder_affinity.fit_transform(train_affinity)


In [0]:
train_sequence = df2['peptide_sequence']

tokenizer = Tokenizer(lower=False, char_level=True)
tokenizer.fit_on_texts(train_sequence)

train_sequence = tokenizer.texts_to_sequences(train_sequence)
train_sequence = pad_sequences(train_sequence, maxlen=32)
# max(list(map(len, train_sequence)))

# np.amax(train_sequence)


In [0]:




print(train_sequence.shape)
print(train_tissue.shape)
print(train_class.shape)
print(train_name.shape)
print(train_affinity.shape)


(449405, 32)
(449405, 30)
(449405, 1)
(449405, 81)
(449405, 1)


In [0]:
train_x = np.concatenate((train_sequence, 
                          train_tissue, 
                          train_class, 
                          train_name, 
                          train_affinity), 
                         axis=1)

In [0]:
def create_generator(noise_dim, out_dim):
    generator = Sequential()

    generator.add(Dense(256,input_dim = noise_dim))
    generator.add(LeakyReLU(0.2));

    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2));

    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(out_dim))
    generator.add(LeakyReLU(0.2))

    generator.compile(loss='binary_crossentropy',
                      optimizer = optimizer)

    return generator

In [0]:
def create_descriminator(input_dimension):
    discriminator = Sequential()
     
    discriminator.add(Dense(1024, input_dim=input_dimension))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', 
                          optimizer=optimizer, 
                          metrics=['accuracy'])
    return discriminator

In [0]:
 
output_dim=145
discriminator = create_descriminator(output_dim)
generator = create_generator(noise_dim, output_dim)

discriminator.trainable = False

gan_input = Input(shape=(noise_dim,))
fake_sequence = generator(gan_input)

gan_output = discriminator(fake_sequence)

gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Display images, and save them if the epoch number is specified
def show_images():
    noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
    generated_images = generator.predict(noise)
    
    
    for i, image in enumerate(generated_images):
        image = tokenizer.sequences_to_texts(train_x)
        print(image)


In [0]:

# batches = train_x.shape[0] / batch_size

for epoch in range(epochs):
    for batch in range(200):
        for _ in range(6):

            noise = np.random.normal(0, 1, size=(batch_size, noise_dim))

            real_x = train_x[np.random.randint(0, train_x.shape[0], size=batch_size)]
            fake_x = generator.predict(noise)

            x = np.concatenate((real_x, fake_x))

            disc_y = np.zeros(2*batch_size)
            disc_y[:batch_size] = 0.95



            discriminator.train_on_batch(x, disc_y)
        for _ in range(2):
            noise = np.random.normal(0, 1, size=(batch_size, noise_dim))

            y_gen = np.ones(batch_size)
            gan.train_on_batch(noise, y_gen)
    print(f'Epoch: {epoch}')
    # print(f'Epoch: {epoch} \t Discriminator Loss: {d_loss} \t\t Generator Loss: {g_loss}')
    #if epoch % 10 == 0:
        #show_images()
generator.save('/content/drive/My Drive/FinalProject822/peptide_data/generator.h5')
gan.save('/content/drive/My Drive/FinalProject822/peptide_data/gan.h5')
discriminator.save('/content/drive/My Drive/FinalProject822/peptide_data/discriminator.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30
Epoch: 31
Epoch: 32
Epoch: 33
Epoch: 34
Epoch: 35
Epoch: 36
Epoch: 37
Epoch: 38
Epoch: 39
Epoch: 40
Epoch: 41
Epoch: 42
Epoch: 43
Epoch: 44
Epoch: 45
Epoch: 46
Epoch: 47
Epoch: 48
Epoch: 49
Epoch: 50
Epoch: 51
Epoch: 52
Epoch: 53
Epoch: 54
Epoch: 55
Epoch: 56
Epoch: 57
Epoch: 58
Epoch: 59
Epoch: 60
Epoch: 61
Epoch: 62
Epoch: 63
Epoch: 64
Epoch: 65
Epoch: 66
Epoch: 67
Epoch: 68
Epoch: 69
Epoch: 70
Epoch: 71
Epoch: 72
Epoch: 73
Epoch: 74
Epoch: 75
Epoch: 76
Epoch: 77
Epoch: 78
Epoch: 79
Epoch: 80
Epoch: 81
Epoch: 82
Epoch: 83
Epoch: 84
Epoch: 85
Epoch: 86
Epoch: 87
Epoch: 88
Epoch: 89
Epoch: 90
Epoch: 91
Epoch: 92
Epoch: 93
Epoch: 94
Epoch: 95
Epoch: 96
Epoch: 97
Epoch: 98
Epoch: 99
Epoch: 100

In [0]:
noise = np.random.normal(0, 1, size=(1, noise_dim))
generated_sequence = generator.predict(noise)
generated_sequence = generated_sequence.astype(int)
"""
inverse_transform(train_x_2[:100])
train_x = np.concatenate((train_sequence, 
                          train_tissue, 
                          train_class, 
                          train_name, 
                          train_affinity), 
                         axis=1)
encoder_class, encoder_name, encoder_affinity, encoder_tissue
peptide_sequence	tissue_name	hla_type_class	hla_type_name	predicted_affinity
(449405, 32)
(449405, 30)
(449405, 1)
(449405, 81)
(449405, 1)
"""
generated_sequence

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  9, 11,  8,  3,  8,  1, 16,  6,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0]])

In [0]:
a = train_sequence.shape[1]
b = a + train_tissue.shape[1]
c = b + train_class.shape[1]
d = c + train_name.shape[1]
e = d + train_affinity.shape[1]

print(generated_sequence[0][:a])


generated_text = tokenizer.sequences_to_texts(generated_sequence[0][:a])
tissue = encoder_tissue.inverse_transform(generated_sequence[0][a:b])
type_class = encoder_class.inverse_transform(generated_sequence[0][b:c])
type_name = encoder_name.inverse_transform(generated_sequence[0][c:d])
affinity = encoder_affinity.inverse_transform(generated_sequence[0][d:])

print({
    'peptide_sequence':generated_text,
    'tissue_name':tissue,
    'hla_type_class':type_class,
    'hla_type_name':type_name,
    'predicted_affinity':type_class
})

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  9
 11  8  3  8  1 16  6  0]


TypeError: ignored